In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta

from pathlib import Path
db_function = str(Path.cwd().parent.parent / "function")
file_path = str(Path.cwd().parent.parent / "files")
fetch_path = file_path + r"\function"

import sys
sys.path.append(db_function)
sys.path.append(file_path)
sys.path.append(fetch_path)

from dotenv import load_dotenv

from access_db import access_db
from get_cols import get_cols
from round_float64_columns import round_float64_columns
from adjust_day_for_temp import adjust_day_based_on_tm_and_hour

import os

import requests

# file fetch
from get_temperature import get_temperature
from wide_temperature import wide_temperature

# github url
from github_upload_url import main

# create md
from create_md import create_md

# telegram
from Telegrambot import TelegramBot

import glob

from time import sleep

In [ ]:
load_dotenv()

In [ ]:
# 🔹 특정 경로의 모든 .md 파일 삭제
md_files = list(Path.cwd().glob("*.md"))

if not md_files:
    print("📂 md 파일이 없습니다. 코드를 실행하지 않습니다.")
else:
    for md_file in md_files:
        try:
            os.remove(md_file)
            print(f"🗑️ 삭제 완료: {md_file}")
        except Exception as e:
            print(f"❌ 삭제 실패: {md_file}, 오류: {e}")

In [2]:
exist_temperature_files = list(Path(file_path).glob("*.xlsx"))

In [22]:
import glob
# 🔹 특정 경로의 모든 .md 파일 삭제
md_files = list(Path(os.getcwd()).glob("*.md"))

for md_file in md_files:
    try:
        os.remove(md_file)
        print(f"🗑️ 삭제 완료: {md_file}")
    except Exception as e:
        print(f"❌ 삭제 실패: {md_file}, 오류: {e}")

🗑️ 삭제 완료: e:\OneDrive\Github\[i-MAES]\[Collaborate]\[Workspace]\000. github_page\Inundata.github.io\_posts\upload-data\2025-02-03-temp.md


In [18]:
md_files

[]

In [13]:
import os

print(f"📂 현재 작업 디렉토리: {os.getcwd()}")
print(f"📂 디렉토리 내 파일 목록: {os.listdir(os.getcwd())}")


📂 현재 작업 디렉토리: e:\OneDrive\Github\[i-MAES]\[Collaborate]\[Workspace]\000. github_page\Inundata.github.io\_posts\upload-data
📂 디렉토리 내 파일 목록: ['2025-02-03-temp.md', 'db-pull.bat', 'db-pull.ipynb', 'db-pull.py']


In [2]:
load_dotenv()

True

In [3]:
host = os.getenv("HOST")
user = os.getenv("USER")
pw = os.getenv("PW")
target_db = os.getenv("iMAES_DB")

In [4]:
cur, conn = access_db(host, user, pw, target_db)

# temperature cols
target_table = "temperature"
temperature_cols = get_cols(cur, target_table)

# fetch `temperature table`
get_temperature(cur, temperature_cols, round_float64_columns)

Temperature table finished


In [5]:
# temperature file
temp_fname = [v for v in os.listdir(file_path) if v.startswith("temperature")][0]
wide_temp_fname = [v for v in os.listdir(file_path) if v.startswith("temperature_wide")][0]

# create wide temp.xlsx file
wide_temperature(file_path, temp_fname)

In [6]:
# temperature file
temp_fname = [v for v in os.listdir(file_path) if v.startswith("temperature")][0]
temp_wide_fname = [v for v in os.listdir(file_path) if v.startswith("temperature_wide")][0]

file_list = [temp_fname, temp_wide_fname]
download_links = main(temp_fname, file_path, file_list)

🔹 기존 릴리즈 삭제 중...
🗑️ 기존 릴리즈 삭제 완료: v1.0 (ID: 198036024)
🔹 기존 릴리즈 없음 → 새 릴리즈 생성 중...
✅ 파일 업로드 성공! 다운로드 링크: https://github.com/Inundata/Inundata.github.io/releases/download/v1.0/temperature_250203.xlsx
✅ 파일 업로드 성공! 다운로드 링크: https://github.com/Inundata/Inundata.github.io/releases/download/v1.0/temperature_wide_250203.xlsx

📥 데이터 다운로드  경로: ['https://github.com/Inundata/Inundata.github.io/releases/download/v1.0/temperature_250203.xlsx', 'https://github.com/Inundata/Inundata.github.io/releases/download/v1.0/temperature_wide_250203.xlsx']


In [23]:
title = f"{datetime.today().strftime('%Y-%m-%d')}기준 기온 데이터"
content = f"""

{datetime.today().strftime('%Y-%m-%d')}기준 기온 데이터입니다.

1. [Long version]({download_links[0]})
2. [Wide version]({download_links[1]})
"""
create_md(title, content, os.getcwd())

md 파일 생성 완료


In [71]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', None)

stnIds = [108, 159, 143, 133, 156, 112, 114, 119]

diff_dict = {}
for stnId in stnIds:
    new = pd.read_excel(r"E:\OneDrive\Github\[i-MAES]\[Collaborate]\[Workspace]\000. github_page\Inundata.github.io\files\temperature_wide_250204.xlsx", sheet_name = f"temp{stnId}1h")
    old = pd.read_excel(r"E:\OneDrive\Github\[i-MAES]\★KOGAS\1. Program\1. MGDRI\python\MGDRI_v1.0\Data_Management\Temperature\Temperature.xlsx", sheet_name = f"temp{stnId}p1h", header = None)

    new.drop(columns = ["cat"], inplace = True)

    old.columns = new.columns

    old["date"] = old.apply(lambda x: datetime(int(x["year"]), int(x["month"]), int(x["day"])), axis = 1)
    new["date"] = new.apply(lambda x: datetime(int(x["year"]), int(x["month"]), int(x["day"])), axis = 1)

    new = new.loc[new["date"].isin(old["date"].values)]

    new.drop(columns = ["date"], inplace = True)
    old.drop(columns = ["date"], inplace = True)

    hour_cols = [f"hour{v}" for v in range(1, 25)]

    new[hour_cols] = new[hour_cols].round(2)
    old[hour_cols] = old[hour_cols].round(2)

    other_cols = ["rc_code", "year", "month", "day"]

    new[other_cols] = new[other_cols].astype(int)
    old[other_cols] = old[other_cols].astype(int)

    diff_new = new.loc[~np.all(new == old, axis = 1)]
    diff_old = old.loc[~np.all(new == old, axis = 1)]

    mask = diff_new != diff_old

    diff_dict[stnId] = [diff_new, diff_old, mask]

In [81]:
for stnId in stnIds:
    print(f"{stnId} : {(diff_dict[stnId][0] != diff_dict[stnId][1]).values.sum()}")

108 : 6
159 : 6
143 : 6
133 : 4
156 : 5
112 : 3
114 : 4
119 : 2
